# Libraries 📚

In [17]:
import random
import numpy as np
import easydict
from glob import glob
import os
import pandas as pd
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import torch.utils.data as data
from tqdm import tqdm
from natsort import natsorted
from glob import glob
import re
import time
import datetime
import warnings
import tensorboardX as tbx
import csv

warnings.simplefilter('ignore')

# Random seed🌱

In [18]:
#再現性の確保
seed = 44
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    #Disabling the benchmark by CUDA convolution operation(GPUを使うときの再現性の担保) (https://pytorch.org/docs/stable/notes/randomness.html)
    torch.backends.cudnn.benchmark = False 
    torch.backends.cudnn.deterministic = True

# Arguments 📗

In [19]:
args = easydict.EasyDict({
    "batch_size":32, # 何枚同時にモデルを画像に入力するか
    "epochs":100000, #アーリーストッピングの
    "learning_rate" : 0.0001, # 損失が下がりきらない場合は下げるといいかも？　adamのデフォルト0.001
    "early_stop_patience" : 25,

    "label_path" : r"E:\2023_3_chuouseiki_WorkingDistance\resize224\L*mm\GroundTruth_ZeroEdge_Cutoff10%.csv", #正解ラベルcsvファイル
    
    "model_save_path" : "Checkpoints",
    "model_file_name" : "model_data40000",
    
    "train_size" : 40000,
    "valid_size" : 10000,
    
    #損失関数のパラメータ
    "tau" : 0.5, # 温度τが小さいほど正・負のサンプル間の差が大きくなり、τが大きくなると、正の類似度は1に近づき、正・負のサンプル間の差は小さくなります。
    "tau2" : 0.5, #小さいとコントラストが付きやすくなるが，無限に発散することに注意.
    "gamma" : 1, #回帰損失を重視する割合,
    "note": "コントラスト:回帰=?:1",
})

# GPU 📺

In [20]:
def check_GPU():
    """
    使用するデバイスを出力する関数

    Returns
    ----------
    device : object
        GPUが使えるなら'cuda:0',使えないなら'cpu'を返す
    """
    print("Check GPU")
    if torch.cuda.is_available():
        print(f"You can use GPU({torch.cuda.get_device_name()})")
        d_type = "cuda:0"
    else:
        print("You use cpu")
        d_type = "cpu"
    print("-----")
    device = torch.device(d_type)
    return device 

# Image & Label 📂

In [21]:
def find_closest_divisible(a:int, b:int):
    """
    バッチサイズで割り切れるデータ数を求めるための関数
    """
    remainder = a % b
    if remainder == 0:
        return int(a)  # aはすでにbで割り切れる
    elif remainder >= b / 2:
        return int(a + (b - remainder))  # aを増やしてbの次の倍数にする
    else:
        return int(a - remainder)  # aを減らしてbの現在の倍数にする

In [22]:
def image_label_collector(args=args):

    """
    画像とラベルを集める関数

    Returns
    ----------
    dataset_dict:dict
    """

    paths = natsorted(glob(args.label_path))
    df = pd.DataFrame()
    for path in tqdm(paths, desc="preparing data...", leave=False):
        df_ = pd.read_csv(path)
        # データフレームにWD追加
        l = int(re.findall("L(\d+)mm",path)[0])
        df_["l"] = l
        # 揺らぎ量(水平＋垂直)
        df_["d"] = np.sqrt(df_["dx"]**2 + df_["dy"]**2) 
        # 揺らぎ量順にソート
        df_.sort_values(by="d", inplace=True)
        # 結合
        df = pd.concat([df,df_])
    df.reset_index(drop=True, inplace=True)

    # データ２分割
    df1 = df[::2]
    df2 = df[1::2]

    # 全データ数
    all_data_num = len(df1)
    # WDごとにグループ分けしたときの1グループあたりのデータ数
    each_group_num = int(all_data_num//(args.batch_size/2))
    # 余り
    reminder_num = int(all_data_num%(args.batch_size/2))
    # 余り分を近距離データから削除
    df1 = df1[reminder_num:].reset_index(drop=True)
    df2 = df2[reminder_num:].reset_index(drop=True)
    # WDごとにグループ分け
    df1["l_group"] = 0
    df2["l_group"] = 0
    for i in range(int(args.batch_size/2)):
        df1["l_group"][each_group_num*i:each_group_num*(i+1)] = i
        df2["l_group"][each_group_num*i:each_group_num*(i+1)] = i

    #　データ数をバッチサイズで割り切れる数にする
    train_data_num = find_closest_divisible(args.train_size, int(args.batch_size/2))
    valid_data_num = find_closest_divisible(args.valid_size, int(args.batch_size/2))

    # 訓練データ抽選
    train_df1 = df1.sample(train_data_num,random_state=seed)
    train_df2 = df2.sample(train_data_num,random_state=seed)
    # サンプルした行のインデックスを取得.df1とdf2はindexが一致してる
    sampled_indexes = train_df1.index
    # 元のDataFrameからサンプルした行を削除
    df1 = df1.drop(sampled_indexes)
    df2 = df2.drop(sampled_indexes)
    # 検証データ抽出
    valid_df1 = df1.sample(valid_data_num,random_state=seed)
    valid_df2 = df2.sample(valid_data_num,random_state=seed)
    # サンプルした行のインデックスを取得.df1とdf2はindexが一致してる
    sampled_indexes = valid_df1.index
    # 元のDataFrameからサンプルした行を削除
    df1 = df1.drop(sampled_indexes)
    df2 = df2.drop(sampled_indexes)
    # データフレームをシャッフルし、シード値を固定
    df1 = df1.sample(frac=1, random_state=seed)
    df2 = df2.sample(frac=1, random_state=seed)

    # 交互に(args.batch_size/2)行ずつデータを取り出し、結合する
    train_df = pd.DataFrame()
    for i in tqdm(range(0, len(train_df1), int(args.batch_size/2)), desc="sorting train data...", leave=False):
        train_df = pd.concat([train_df, train_df1[i:i+int(args.batch_size/2)], train_df2[i:i+int(args.batch_size/2)]])
    valid_df = pd.DataFrame()
    for i in tqdm(range(0, len(valid_df1), int(args.batch_size/2)), desc="sorting valid data...", leave=False):
        valid_df = pd.concat([valid_df, valid_df1[i:i+int(args.batch_size/2)], valid_df2[i:i+int(args.batch_size/2)]])
    # テストデータはそのまま結合
    test_df = pd.concat([df1,df2])

    # 辞書型にまとめる．
    dataset_dict = {
        "train" : train_df,
        "valid" : valid_df,
        "test" : test_df,
    }

    # 保存
    for key, value in dataset_dict.items():
        value.to_csv(f"{key}.csv",index=False)
    
    return dataset_dict

# Data Normalization 📊

In [23]:
def image_normalization(train_df, num=500):
    """
    画像の標準化を行うためにデータの平均と標準偏差を求める関数

    Parameters
    ----------
    img_path_list : list
        globで集めた画像path
    num : int
        平均と標準偏差を求めるのに使う画像枚数 

    Returns
    ----------
    mean : tuple
        データセットの画素の平均
    std : tuple
        データセットの画素の標準偏差    
    """

    tensors = []
    to_tensor = transforms.ToTensor()
    sample_df = train_df.sample(num)
    img_path_list = sample_df["fname"]
    for img in img_path_list:
        img = Image.open(img)
        img = to_tensor(img)
        tensors.append(img)
    tensors = torch.stack([img_t for img_t in tensors], dim=3) #dimはどこでもよい
    mean = tensors.view(1,-1).mean(dim=1).item()
    std = tensors.view(1,-1).std(dim=1).item()
    
    df = pd.DataFrame({"mean":[mean],
                       "std":[std]})
    df.to_csv("mean_std.csv")
    
    return mean, std    

# Data set 🧰

In [24]:
class ImageTransform:
    """
    画像の前処理クラス
    訓練時だけ、データオーギュメンテーション(DA)ができるように、train,validで分けて書いた

    Attributes
    ----------
    mean : int
        データセットの画素値の平均値
    std : int
        データセットの画素値の標準偏差
    """   

    def __init__(self, mean, std):
        self.data_transform = {
            'train':transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean, std) 
            ]),
            'valid':transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
    
    #引数なしで呼ばれたときの挙動を定義
    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

In [25]:
class Image_Dataset(data.Dataset):
    """
    画像のデータセットクラス。
    Pytorch Dataset class を継承

    Attributes
    ----------
    file_list : list
        画像のpath_list
    label_list : list
        ラベルのpath_list       
    transform : object
        class ImageTransform()    
    phase : str
        'train' or 'valid'
    """

    def __init__(self, df, transform, phase='train'):
        self.file_list = df["fname"] 
        self.labels = df[["dx","dy"]].to_numpy()
        #self.l = df["l"]
        self.d = df["d"]
        self.transform = transform  
        self.phase = phase  

    #このクラスの画像枚数を定義。
    def __len__(self):
        return len(self.file_list) 

    #このクラスに角括弧でアクセスした時の挙動を定義
    def __getitem__(self, index):
        # 画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)
        #img = img.convert("L") #グレイスケール 
        # 前処理
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([width, height]) 

        #label
        label = self.labels[index]
        #Tensorに変換
        label = torch.tensor(label, dtype=torch.float32) 

        # 空気揺らぎ量
        d = self.d[index]           

        return img_transformed, label, d

# NetWork 🧠

In [26]:
#モデル

# # 残差ブロック
class block(nn.Module):
    def __init__(self, first_conv_in_channels, first_conv_out_channels, identity_conv=None, stride=1):
        """
        残差ブロックを作成するクラス
        Args:
            first_conv_in_channels : 1番目のconv層（1×1）のinput channel数
            first_conv_out_channels : 1番目のconv層（1×1）のoutput channel数
            identity_conv : channel数調整用のconv層
            stride : 3×3conv層におけるstide数。sizeを半分にしたいときは2に設定
        """        
        super(block, self).__init__()

        # 1番目のconv層（1×1）
        self.conv1 = nn.Conv2d(first_conv_in_channels, first_conv_out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(first_conv_out_channels)

        # 2番目のconv層（3×3）
        # パターン3の時はsizeを変更できるようにstrideは可変
        self.conv2 = nn.Conv2d(first_conv_out_channels, first_conv_out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(first_conv_out_channels)

        # 3番目のconv層（1×1）
        # output channelはinput channelの4倍になる
        self.conv3 = nn.Conv2d(first_conv_out_channels, first_conv_out_channels*4, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(first_conv_out_channels*4)
        self.relu = nn.ReLU()

        # identityのchannel数の調整が必要な場合はconv層（1×1）を用意、不要な場合はNone
        self.identity_conv = identity_conv


    def forward(self, x):

        identity = x.clone()  # 入力を保持する

        x = self.conv1(x)  # 1×1の畳み込み
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)  # 3×3の畳み込み（パターン3の時はstrideが2になるため、ここでsizeが半分になる）
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)  # 1×1の畳み込み
        x = self.bn3(x)

        # 必要な場合はconv層（1×1）を通してidentityのchannel数の調整してから足す
        if self.identity_conv is not None:
            identity = self.identity_conv(identity)
        x += identity

        x = self.relu(x)

        return x
    
#  Resnet50
class ResNet(nn.Module):
    def __init__(self,block):
        super(ResNet,self).__init__()

        # conv1はアーキテクチャ通りにベタ打ち
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # in:(64,112*112)、out:(64,56*56)
        self.maxpool = nn.Identity()

        # conv2_xはサイズの変更は不要のため、strideは1
        self.conv2_x = self._make_layer(block, 3, res_block_in_channels=64, first_conv_out_channels=64, stride=1)

        # conv3_x以降はサイズの変更をする必要があるため、strideは2
        self.conv3_x = self._make_layer(block, 4, res_block_in_channels=256,  first_conv_out_channels=128, stride=2)
        self.conv4_x = self._make_layer(block, 6, res_block_in_channels=512,  first_conv_out_channels=256, stride=2)
        self.conv5_x = self._make_layer(block, 3, res_block_in_channels=1024, first_conv_out_channels=512, stride=2)

        #self.avgpool = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        # self.fc1 = nn.Linear(512*4,2)
        self.fc1 = nn.Identity()
    
        # mlp projection head
        self.projection = nn.Sequential(
            nn.Linear(in_features=2048, out_features=2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=128),
            nn.BatchNorm1d(128),
        )
        # Regression
        self.regression = nn.Sequential(
            nn.Linear(2048, 512), 
            nn.LeakyReLU(),       
            nn.Linear(512, 128),  
            nn.LeakyReLU(),
            nn.Linear(128, 32),        
            nn.Linear(32, 2)
        )

    def forward(self,x):

        x = self.conv1(x)  
        x = self.bn1(x)     
        x = self.relu(x)    
        x = self.maxpool(x) #out:(64*16*16)
        

        x = self.conv2_x(x)  # in:(64,56*56)  、out:(256,56*56)
        x = self.conv3_x(x)  # in:(256,56*56) 、out:(512,28*28)
        x = self.conv4_x(x)  # in:(512,28*28) 、out:(1024,14*14)
        x = self.conv5_x(x)  # in:(1024,14*14)、out:(2048,7*7)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1) # 2048
        x = self.fc1(x)
        #backbone_output = x.clone()

        projection_output = self.projection(x)
        regression_output = self.regression(x)

        return projection_output, regression_output

    def _make_layer(self, block, num_res_blocks, res_block_in_channels, first_conv_out_channels, stride):
        layers = []

        # 1つ目の残差ブロックではchannel調整、及びsize調整が発生する
        # identifyを足す前に1×1のconv層を追加し、サイズ調整が必要な場合はstrideを2に設定
        identity_conv = nn.Conv2d(res_block_in_channels, first_conv_out_channels*4, kernel_size=1,stride=stride)
        layers.append(block(res_block_in_channels, first_conv_out_channels, identity_conv, stride))

        # 2つ目以降のinput_channel数は1つ目のoutput_channelの4倍
        in_channels = first_conv_out_channels*4

        # channel調整、size調整は発生しないため、identity_convはNone、strideは1
        for i in range(num_res_blocks - 1):
            layers.append(block(in_channels, first_conv_out_channels, identity_conv=None, stride=1))

        return nn.Sequential(*layers)

# Similar matrix 🎭

In [27]:
# 類似行列
def sim_matrix(output1:torch.tensor, output2:torch.tensor):

    """
    Attributes
    ----------
    output1 : torch.tensor
        ネットワークの出力前半
    output2 : torch.tensor
        ネットワークの出力後半
        
    Returns
    ----------
    sim : torch.tensor
        類似行列
    """   
    # 実は2つに分ける意味ない
    zi_norm = F.normalize(output1, dim=1)
    zj_norm = F.normalize(output2, dim=1)
    representation = torch.cat([zi_norm,zj_norm], dim=0)
    sim = torch.matmul(representation, torch.t(representation))
    return sim

# Loss function 🌈

In [28]:
def contrast_loss_function(sim, ds, device, args=args):
    harf_batch_size = int(args.batch_size/2)

    # 行列全体にexp(./tau)をかける
    sim = torch.exp(sim/args.tau)
    
    # 重み
    # 空気揺らぎ量の差
    ds = ds.to(torch.float)
    matrix1 = ds.repeat(args.batch_size,1)
    matrix2 = matrix1.T
    weight = torch.abs(matrix2 - matrix1)
    # たまたま揺らぎ量が重複する場合，0の要素を1に変換します.
    weight[weight == 0] = 1
    # 指数にする
    weight = torch.exp(weight/args.tau2)
    # 指示関数
    mask = (~torch.eye(args.batch_size, args.batch_size, dtype=bool)).float()
    weight = weight * mask
    weight = weight.to(device)
    bottom = weight*sim

    #対角成分を抽出　positive pairを抽出
    sim_ij = torch.diag(bottom, harf_batch_size)
    sim_ji = torch.diag(bottom, -harf_batch_size)
    top = torch.cat([sim_ij, sim_ji], dim=0)

    all_losses = - torch.log(top/torch.sum(bottom, dim = 1))
    loss = torch.sum(all_losses)/ args.batch_size
    return loss

In [29]:
def regressison_loss_function(output, labels, args=args):
    criterion = nn.MSELoss()
    loss = args.gamma * criterion(output, labels) / args.batch_size
    return loss

# Train 🏋️‍♂️

In [30]:
def train(model, dataloaders_dict, device, opt, writer, earlystopping):

    #global epoch

    BREAK = False
    print(f"\nStarting training")
    for epoch in range(0, args.epochs+1):

        # epochごとの学習と検証のループ
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # モデルを訓練モードに
            else:
                model.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_contrast_loss = 0.0
            epoch_regression_loss = 0.0                

            #未訓練時の性能評価
            if (epoch == 0 and phase=='train'):
                continue
            
            # データローダーからミニバッチを取り出すループ。args['batch_size']枚ごと取り出す
            for inputs, labels, ds in tqdm(dataloaders_dict[phase], leave=False, desc='Epoch {}/{} {}'.format(epoch, args.epochs, phase)): 

                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                opt.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    projection_output, regression_output = model(inputs)
                    output1, output2 = torch.chunk(projection_output, chunks=2, dim=0)
                    sim = sim_matrix(output1, output2)
                    contrast_loss = contrast_loss_function(sim, ds, device)
                    regression_loss = regressison_loss_function(regression_output, labels)
                    loss = contrast_loss + regression_loss

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward() 
                        opt.step()

                # 損失の計算
                epoch_loss += loss.item()
                epoch_contrast_loss += contrast_loss.item()
                epoch_regression_loss += regression_loss.item()

            #tensorboardに出力
            if phase == 'train':
                writer.add_scalar('train_epoch_loss', epoch_loss, epoch) #(グラフ名, y座標, x座標) 
                writer.add_scalar('train_contrast_epoch_loss', epoch_contrast_loss, epoch)    
                writer.add_scalar('train_epoch_regression_loss', epoch_regression_loss, epoch)           
            elif phase == 'valid':
                writer.add_scalar('valid_epoch_loss', epoch_loss, epoch)
                writer.add_scalar('valid_contrast_epoch_loss', epoch_contrast_loss, epoch)    
                writer.add_scalar('valid_epoch_regression_loss', epoch_regression_loss, epoch)                 
                #毎エポックearlystoppingの判定をさせる
                earlystopping(epoch_loss, model) #callメソッド呼び出し
                if earlystopping.early_stop: #ストップフラグがTrueの場合、breakでforループを抜ける
                    print("Early Stopping!")
                    BREAK = True
        if BREAK:
            break

# EarlyStopping ⛔

In [31]:
class EarlyStopping:
    """
    earlystoppingクラス
    損失が下がったことを判断して学習を打ち切る

    Attributes
    ----------
    patience : int
        何回損失が下がらなかったら学習を打ち切るか
    verbose : bool
    """

    def __init__(self, patience, verbose):
        """引数 : 最小値の非更新数カウンタ、表示設定"""

        self.patience = patience    #設定ストップカウンタ
        self.verbose = verbose      #表示の有無
        self.counter = 0            #現在のカウンタ値
        self.best_score = None      #ベストスコア
        self.early_stop = False     #ストップフラグ
        self.val_loss_min = np.Inf   #前回のベストスコア記憶用

    def __call__(self, val_loss, model):
        """
        特殊(call)メソッド
        実際に学習ループ内で最小lossを更新したか否かを計算させる部分
        """
        score = -val_loss

        if self.best_score is None:  #1Epoch目の処理
            self.best_score = score   #1Epoch目はそのままベストスコアとして記録する
            self.checkpoint(val_loss, model)  #記録後にモデルを保存してスコア表示する
        elif score < self.best_score:  # ベストスコアを更新できなかった場合
            self.counter += 1   #ストップカウンタを+1
            # if self.verbose:  #表示を有効にした場合は経過を表示
            #     print(f'EarlyStopping counter: {self.counter} out of {self.patience}')  #現在のカウンタを表示する 
            if self.counter >= self.patience:  #設定カウントを上回ったらストップフラグをTrueに変更
                self.early_stop = True
        else:  #ベストスコアを更新した場合
            self.best_score = score  #ベストスコアを上書き
            self.checkpoint(val_loss, model)  #モデルを保存してスコア表示
            self.counter = 0  #ストップカウンタリセット

    def checkpoint(self, val_loss, model):
        '''ベストスコア更新時に実行されるチェックポイント関数'''
        if self.verbose:  #表示を有効にした場合は、前回のベストスコアからどれだけ更新したか？を表示
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        save_path = os.path.join(args.model_save_path, f'{args.model_file_name}.pth')
        torch.save(model.state_dict(), save_path)  #ベストモデルを指定したpathに保存
        self.val_loss_min = val_loss  #その時のlossを記録する

# Run Main 🏃‍♀️

In [32]:
# 時間計測開始
time_start = time.time()

#GPU
device = check_GPU()

os.makedirs(args.model_save_path, exist_ok=True) #重みファイルの保存ディレクトリ
# 学習条件の保存
csv_file = os.path.join(args.model_save_path,f"{args.model_file_name}_info.csv")
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    saver = csv.writer(file)
    # ヘッダー（辞書のキー）を書き込む
    saver.writerow(args.keys())
    # データ（辞書の値）を書き込む
    saver.writerow(args.values())


#train test split  
#dataset_dict = image_label_collector()
    
train_size = find_closest_divisible(args.train_size, int(args.batch_size))
valid_size = find_closest_divisible(args.valid_size, int(args.batch_size))

# Quick start (train test splitをコメントアウトして使う)
train_df = pd.read_csv("train.csv")[:train_size]
valid_df = pd.read_csv("valid.csv")[:valid_size]
test_df = pd.read_csv("test.csv")
dataset_dict = {
        "train":train_df,
        "valid":valid_df,
        "test":test_df
    }

#画像の標準化
mean, std = image_normalization(dataset_dict["train"])

transform = ImageTransform(mean, std)

#logger
writer = tbx.SummaryWriter()

# Dataset
train_dataset = Image_Dataset(df=dataset_dict["train"], transform=transform, phase='train') 
valid_dataset = Image_Dataset(df=dataset_dict["valid"], transform=transform, phase='valid')

# make dataloader
train_dataloader=torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    pin_memory=True,
    shuffle=False,
    )

valid_dataloader=torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=args.batch_size,
    pin_memory=True,
    shuffle=False,
    )

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "valid": valid_dataloader}          

#Network
model = ResNet(block)
model.to(device)

#optimizer
opt = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

#early topping 
earlystopping = EarlyStopping(patience=args.early_stop_patience, verbose=True) 

#train
train(model, dataloaders_dict, device, opt, writer, earlystopping)

writer.close()

#時間計測終了
time_end = time.time()
#経過時間（秒）
elapsed = time_end - time_start
#秒➡時間
td = datetime.timedelta(seconds=elapsed)
#経過時間記憶
f = open('elapsed_time.txt','a')
f.write(f"{args.model_file_name}.pth : {str(td)} ,")
f.close()

Check GPU
You can use GPU(NVIDIA GeForce RTX 3090)
-----



Starting training


Validation loss decreased (inf --> 5625.198720).  Saving model ...


Validation loss decreased (5625.198720 --> 4910.085599).  Saving model ...


Validation loss decreased (4910.085599 --> 4907.389169).  Saving model ...


...

Validation loss decreased (4692.345592 --> 4653.800603).  Saving model ...


Validation loss decreased (4653.800603 --> 4644.607992).  Saving model ...


Validation loss decreased (4644.607992 --> 4644.354204).  Saving model ...


Validation loss decreased (4644.354204 --> 4639.106564).  Saving model ...


Early Stopping!


#   
    　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　{ヽﾐ∧
    　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　 　 彡彡 　 .＼
    　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　 　 彡彡　　　 .● ヽ
    　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　 彡　　 （　　　　＼
    　　　　　　　　　　　　　　　　 ＿───────────＿＿＿＿──../　             /ヽ 　 ヽ　　 ／￣￣￣￣￣￣￣￣
    　　　　　　　　　　　　　　　／　　　 　 　 /　 　 　 ./　　　　　　 　 ./　　ヽ　o丿　＜　ワープに使ってください
    　 　 　 　 　 　 　 彡彡彡　 　 　 　 　 ./　　　　　/　　　　　 　 　  │ 　 　 　 　 　＼＿＿＿＿＿＿＿＿
    　　 　 　 　 　 彡彡彡彡　　　 　 　 　  |ｺﾞｰﾙﾄﾞｼｯﾌﾟ/　　　　　　 　 　 │
    　　 　 　 彡彡彡　　　 /　　　　　　　 ..￣￣￣￣￣　 .ヽ、　 　 　 ヽ　ﾉ
    　　 　 彡彡　　　　 　 ﾉ　 　 　 ／￣￣｀ ヽ ､　　 　 　 　 ｀､　　 　 ﾉ　＼
    　　　 彡　　　　　　 ／　　 ／　｀､　　　／　　 ｀　ー ､ ＿＿ヽ　　 ヽ　 　  ヽ 、
    　　　　　 　 　 　 ／ 　 ／　　　 /　　 /　　　　　　　　 　 　 　 ＼　 丶　- ､ ｀､
    　　　　　　　　　/　.／　　　　／ 　 ／　　　　　　　　　　　　　　　 ＼　｀､　　＼＼
    　　　　 　 　 .／.／　　　　　 |　　/　　　　　　　　　　　　　　　 　  ＼ 丶 　 　ヽ ヽ
    　　　 　 ___／.／　 　 　 　 　 |　|　　　　　　　　　　　　　　　　　　　ヽ｀､ 　 　｀､｀､
    　　　　/ |__／　　　　　　　　　 | |　　　　　　　　　　　　　　　　　　　　｀､ヽ、　　 ｔﾆゝ
    　　 　 ￣　　 　 　 　 　 　 """"''""'"" 　　　　　　　　　　　　　 　 　 　 ヽ ､ヽ　　　
    　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　 　 　 　 　 　 ｔﾆゝ

以下のコマンドをターミナルに入力して，ログを見る  
conda activate env_pytorch  
tensorboard --logdir runs  
終わるとき➡ctrlキーとcを同時に入力  